In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnchoredText
import mplhep as hep
import pickle
from coffea.util import load
from coffea.hist import plot
import re

In [2]:
lumi_weights = None
with open("filefetcher/aram_samples.scales.pkl", "rb") as fin:
    lumi_weights = pickle.load(fin)
    lumi_weights["data_mu"] = 1.0
    lumi_weights["data_el"] = 1.0
output = load("hists_wpt_filefetcher_aram_sample.coffea")

In [3]:
[x for x in lumi_weights.keys() if x[-2:] == "mu"]

['data_mu',
 'wl0_mu',
 'wl1_mu',
 'wl2_mu',
 'ttbar_dilepton_mu',
 'ttbar_onelepton_mu',
 'ttbar_hadronic_mu',
 'ww-mu',
 'wz-mu',
 'zz-mu',
 'ztautau-mu',
 'wtau0-mu',
 'wtau1-mu',
 'wtau2-mu']

In [5]:
from hist import Hist, Stack
data_mu = [h * lumi_weights[name] for name, h in output.items() if name.find("data") > -1 and name[-2:] == "mu"]
mc_mu = [h * lumi_weights[name] for name, h in output.items() if name.find("data") == -1 and name[-2:] == "mu"]
data_el = [h * lumi_weights[name] for name, h in output.items() if name.find("data") > -1 and name[-2:] == "el"]
mc_el = [h * lumi_weights[name] for name, h in output.items() if name.find("data") == -1 and name[-2:] == "el"]


In [ ]:
mts_mu = muon_channel[:, "cent", ...].project("dataset","mt").stack("dataset")
for i in range(len(mts_mu)):
    print("mu", mts_mu[i].name)
mts_mu_scaled = Stack(*(h * lumi_weights[h.name] for h in mts_mu))

mts_el = electron_channel[:, "cent", ...].project("dataset","mt").stack("dataset")
for i in range(len(mts_mu)):
    print("el", mts_el[i].name)
mts_el_scaled = Stack(*(h * lumi_weights[h.name] for h in mts_el))

In [ ]:
f, axs = plt.subplots(1, 1, sharex=True, sharey=True)

hep.histplot([x for x in mts_mu_scaled], label=[x.name for x in mts_mu_scaled])

axs.legend(fontsize=16)

In [ ]:
analysis["wl0_mu", "cent", ...].project("mt") * lumi_weights["wl0_mu"]

In [ ]:
plt.style.use([hep.style.ROOT, {'font.size': 16}])

# plot options for data
data_err_opts = {
    'linestyle': 'none',
    'marker': '.',
    'markersize': 10.,
    'color': 'k',
    'elinewidth': 1,
}

for i in range(0, int(round(len(output.keys())))-1, 2)[2:]:
    fig, ((ax1, ax2),(rax1, rax2)) = plt.subplots(2, 2, figsize=(16, 7), gridspec_kw={"height_ratios": (3, 1)}, sharex=True)
    fig.subplots_adjust(hspace=.07)
    h1name = list(output.keys())[i]
    h2name = list(output.keys())[i+1]
    if any([h1name.startswith('cutflow'), h2name.startswith('cutflow')]): break
    print(h1name, h2name)
    h1 = output[h1name]
    h2 = output[h2name]
    # if True: normalize histo to 1
    dense = False

    for ax, rax, h in zip([ax1, ax2], [rax1, rax2], [h1, h2]):
        # scale MC samples according to lumi (for now, dummy scale)
        scales = {
            'ttbar': 1.0,
            'ttbarprime':1.0
        }
        h.scale(scales,axis='dataset')   
        notdata = re.compile('(?!Data)')
        if hasattr(h, 'dim'):
            plot.plot1d(h[notdata], ax=ax, legend_opts={'loc':1}, density=dense, stack=True, clear=False);
            plot.plot1d(h["Data"], ax=ax, legend_opts={'loc':1}, density=dense, error_opts=data_err_opts, clear=False);
            # now we build the ratio plot
            plot.plotratio(
                   num=h["Data"].sum("dataset"),
                    denom=h[notdata].sum("dataset"),
                    ax=rax,
                    error_opts=data_err_opts,
                    denom_fill_opts={},
                    guide_opts={},
                    unc='num'
                )
        else:
            continue    
        #break
    for ax, rax, hname in zip([ax1, ax2], [rax1, rax2], [h1name, h2name]):
        at = AnchoredText(r"$1\mu, 1e$"+"\n"+
                           "2+ jets"+"\n"+
                          r"$|\eta| < 2.5$",
                           loc=2, frameon=False)
        ax.add_artist(at)
        ax.set_ylim(0.001, None)
        if hname.startswith("btag") or hname.startswith("DeepCSV_trackDecayLenVal"):
            ax.semilogy()
        rax.set_ylabel('Data/Pred.')
        rax.set_ylim(0,2)
        ax.set_xlabel(None)
        
    hep.mpl_magic(ax1)
    hep.mpl_magic(ax2)